# CS 418: Final Cleaned Data

## Group: Neha Anand, Sara Ali-Hasan, Risha Bongu, Whitney Le, Sebastian Sobolewski

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import statistics as st

In [2]:
healthData = pd.read_csv('healthData.csv')

In [3]:
healthData.head()

,_id,ReferenceID,ReportYear,DiagnosisCategory,DiagnosisSubCategory,TreatmentCategory,TreatmentSubCategory,Determination,Type,AgeRange,PatientGender,IMRType,DaysToReview,DaysToAdopt,Findings
0,1,MN25-43524,2025,Autism Spectrum,Autistic Disorder,Autism Related Tx,Appld Behav Analysis,Overturned Decision of Health Plan,Medical Necessity,0 to 10,Female,Standard,11.0,45,Nature of Statutory Criteria/Case Summary: A p...
1,2,MN25-43522,2025,Endocrine/Metabolic,Obesity,Pharmacy,Weight Control,Upheld Decision of Health Plan,Medical Necessity,51 to 64,Female,Standard,20.0,80,Findings: The physician reviewer found that N...
2,3,MN25-43521,2025,Skin Disorders,Psoriasis,Pharmacy,Biologics,Overturned Decision of Health Plan,Medical Necessity,65+,Male,Standard,13.0,56,Findings: The physician reviewer found that A...
3,4,MN25-43520,2025,Digestive System/ GI,Crohn's Disease,Pharmacy,Digestion/GI Rx,Overturned Decision of Health Plan,Medical Necessity,51 to 64,Female,Standard,18.0,54,Nature of Statutory Criteria/Case Summary: The...
4,5,MN25-43519,2025,CNS/ Neuromusc Dis,Other,Acute Med Svc Inpt,Early Discharge,Overturned Decision of Health Plan,Medical Necessity,41 to 50,Female,Standard,14.0,66,Findings: The physician reviewer found that N...


In [4]:
healthData.describe()

#initial overview before we realized there are duplicates and null values, looked at mean, median, mode

,_id,ReportYear,DaysToReview,DaysToAdopt
count,39447.000000,39447.000000,38764.000000,39447.000000
mean,19724.000000,2015.205263,16.314415,37.501559
std,11387.512371,6.067554,10.201466,21.602859
min,1.000000,2001.000000,0.000000,1.000000
25%,9862.500000,2010.000000,7.000000,24.000000
50%,19724.000000,2016.000000,18.000000,36.000000
75%,29585.500000,2020.000000,21.000000,46.000000
max,39447.000000,2025.000000,133.000000,636.000000


In [5]:
duplicate = healthData[healthData.duplicated(subset = ['ReferenceID'])]
print(duplicate)

#shows us what rows had duplicates 

         _id ReferenceID  ReportYear     DiagnosisCategory  \
171      172  MN24-43230        2024   Endocrine/Metabolic   
172      173  MN24-43230        2024   Endocrine/Metabolic   
173      174  MN24-43230        2024   Endocrine/Metabolic   
16581  16582  MN17-26789        2017      Genetic Diseases   
20077  20078  EI16-23294        2016                Cancer   
20900  20901  MN16-22472        2016  Digestive System/ GI   
22426  22427  EI15-20947        2015       Mental Disorder   
22757  22758  MN15-20615        2015        Skin Disorders   
23003  23004  EI15-20370        2015                Vision   
24228  24229  EI14-19144        2014        Not Applicable   
25643  25644  UR13-15728        2013       Mental Disorder   
26186  26187  MN13-15159        2013       Mental Disorder   
27884  27885  MN12-13451        2012                Cancer   
27921  27922  MN12-13415        2012       Autism Spectrum   
27933  27934  MN12-13404        2012                Cancer   
30244  3

In [6]:
healthData.isnull().sum()
#the null values at the beginning

_id                       0
ReferenceID               0
ReportYear                0
DiagnosisCategory         0
DiagnosisSubCategory      2
TreatmentCategory         0
TreatmentSubCategory      1
Determination             0
Type                      0
AgeRange                691
PatientGender           691
IMRType                   0
DaysToReview            683
DaysToAdopt               0
Findings                  0
dtype: int64

In [7]:
healthDataclean = healthData.drop_duplicates(subset = ['ReferenceID'], keep = 'first')
#dropped all duplicates but kept the first occurrance 

In [8]:
def age(group):
    modeAge = group['AgeRange'].mode()
    if not modeAge.empty:
        group['AgeRange'] = group['AgeRange'].fillna(modeAge[0])
    return group

def gender(group):
    modeGender = group['PatientGender'].mode()
    if not modeGender.empty:
        group['PatientGender'] = group['PatientGender'].fillna(modeGender[0])
    return group

def days(group):
    modeDays = group['DaysToReview'].mode()
    if not modeDays.empty:
        group['DaysToReview'] = group['DaysToReview'].fillna(modeDays[0])
    return group


def DSubCategory(group):
    modeSub = group['DiagnosisSubCategory'].mode()
    if not modeSub.empty:
        group['DiagnosisSubCategory'] = group['DiagnosisSubCategory'].fillna(modeSub[0])
    return group

def TSubCategory(group):
    modeTreat = group['TreatmentSubCategory'].mode()
    if not modeTreat.empty:
        group['TreatmentSubCategory'] = group['TreatmentSubCategory'].fillna(modeTreat[0])
    return group

healthDataclean = healthDataclean.groupby(['DiagnosisCategory']).apply(age).reset_index(drop=True)
healthDataclean = healthDataclean.groupby(['DiagnosisCategory', 'AgeRange']).apply(gender).reset_index(drop=True)
healthDataclean = healthDataclean.groupby(['DiagnosisCategory', 'PatientGender']).apply(days).reset_index(drop=True)
healthDataclean = healthDataclean.groupby(['DiagnosisCategory']).apply(DSubCategory).reset_index(drop=True)
healthDataclean = healthDataclean.groupby(['TreatmentCategory']).apply(TSubCategory).reset_index(drop=True)

In [9]:
healthDataclean.isnull().sum()
# did mode imputation and filled in null values, the remaining nulls aren't changed because we aren't using them 

_id                     0
ReferenceID             0
ReportYear              0
DiagnosisCategory       0
DiagnosisSubCategory    0
TreatmentCategory       0
TreatmentSubCategory    0
Determination           0
Type                    0
AgeRange                0
PatientGender           0
IMRType                 0
DaysToReview            0
DaysToAdopt             0
Findings                0
dtype: int64

In [10]:
healthDataclean.describe()
#updated data descriptive statistics after dropping duplicates

,_id,ReportYear,DaysToReview,DaysToAdopt
count,39413.000000,39413.000000,39413.000000,39413.000000
mean,19715.995179,2015.210159,16.377946,37.487149
std,11384.366446,6.064549,10.121379,21.594478
min,1.000000,2001.000000,0.000000,1.000000
25%,9857.000000,2010.000000,7.000000,24.000000
50%,19711.000000,2016.000000,18.000000,36.000000
75%,29575.000000,2020.000000,21.000000,46.000000
max,39447.000000,2025.000000,133.000000,636.000000


In [11]:
healthDataclean.mode().iloc[0]

#mode after dropping duplicates 

_id                                                                     1
ReferenceID                                                     EI01-2846
ReportYear                                                         2016.0
DiagnosisCategory                                    Orth/Musculoskeletal
DiagnosisSubCategory                                                Other
TreatmentCategory                                                Pharmacy
TreatmentSubCategory                                                Other
Determination                          Overturned Decision of Health Plan
Type                                                    Medical Necessity
AgeRange                                                         51 to 64
PatientGender                                                      Female
IMRType                                                          Standard
DaysToReview                                                         21.0
DaysToAdopt                           